<a href="https://colab.research.google.com/github/UIT-21520916/NCKH/blob/task5/Model_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom


import os
import matplotlib.pyplot as plt
import keras

import cv2

In [ ]:
df=pd.read_csv('/home/XSS_dataset.csv', encoding='utf-8-sig')
# with open('/home/XSS_dataset.csv') as df:
#   df = pd.read_csv(df, names=header_list)

In [ ]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [ ]:
df=df[df.columns[-2:]]

In [ ]:
df.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [ ]:
df.shape
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13686 entries, 0 to 13685
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  13686 non-null  object
 1   Label     13686 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 214.0+ KB
                                                Sentence  Label
0      <li><a href="/wiki/File:Socrates.png" class="i...      0
1                   <tt onmouseover="alert(1)">test</tt>      1
2      \t </span> <span class="reference-text">Steeri...      0
3      \t </span> <span class="reference-text"><cite ...      0
4      \t </span>. <a href="/wiki/Digital_object_iden...      0
...                                                  ...    ...
13681             <img onpointerenter=alert(1)>XSS</img>      1
13682  <source onbeforepaste="alert(1)" contenteditab...      1
13683  <div draggable="true" contenteditable>drag me<...      1
13684  <li><cite id="CITEREFDomingos2015" class=

In [ ]:
# Convert to ASCII

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        """Some characters have values very big e.d 8221 adn some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater 
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
                
                
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
                
            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer


In [ ]:
sentences=df['Sentence'].values
sentences[1]

'<tt onmouseover="alert(1)">test</tt>'

In [ ]:


# send each sentence to be converted to ASCII


arr=np.zeros((df.shape[0],100,100)) # 1 mảng gồm len(sentences) mảng (100x100)
arr1=np.zeros((df.shape[0],100,100))
for i in range(len(sentences)): #sentences = df.shape[0]
    
    image=convert_to_ascii(sentences[i]) 

    # arr1[i]=np.array(image,dtype='float')

    x=np.asarray(image,dtype='float') #khi mà image thay đổi thì x cũng thay đổi theo
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    # tại sao phải resize khi mà dsize không thay đổi


    image/=128

    
#     if i==1:
#         plt.plot(image)
#         plt.show()    
    arr[i]=image



In [ ]:
print(arr[3])
# print(arr1[3]) 

[[0.0703125 0.25      0.46875   ... 0.7578125 0.84375   0.25     ]
 [0.90625   0.7890625 0.9375    ... 0.90625   0.9140625 0.890625 ]
 [0.7890625 0.25      0.6171875 ... 0.78125   0.9140625 0.7734375]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [ ]:
print("Input data shape : ", arr.shape)

Input data shape :  (13686, 100, 100)


In [ ]:
# Reshape data for input to CNN
data = arr.reshape(arr.shape[0],1, 100, 100, 1)

In [ ]:
data.shape

(13686, 1, 100, 100, 1)

In [ ]:
y=df['Label'].values

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)
# việc thiết lập random_state giúp đảm bảo tính nhất quán của kết quả khi bạn chạy lại đoạn mã này, và giúp bạn có thể tái sử dụng kết quả chia dữ liệu trước đó để kiểm tra và so sánh với các mô hình học máy khác.

In [ ]:
# import libraries for Making Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

In [ ]:
# A basic CNN Model
# Number of layers = 11
# Number of Convolutional layer: 3

# model=tf.keras.models.Sequential([
    
#     tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
    
# ])
model = tf.keras.models.Sequential([
    tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu), input_shape=(1, 100, 100, 1)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(2, 2)),

    tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(128, (3, 3), activation='relu')),
    tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(2, 2)),

    tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(256, (3, 3), activation='relu')),
    tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(2, 2)),
    
    tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
    
    tf.keras.layers.LSTM(256, return_sequences=False),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 1, 98, 98, 64)    640       
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 1, 49, 49, 64)    0         
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 1, 47, 47, 128)   73856     
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, 1, 23, 23, 128)   0         
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 1, 21, 21, 256)  

In [ ]:
# Stop when validation accuracy > 97

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.97):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [ ]:
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY)
                      #  callbacks=[callbacks]
                     )

Epoch 1/10
86/86 [==============================] - 776s 9s/step - loss: 0.4752 - accuracy: 0.7653 - val_loss: 0.3989 - val_accuracy: 0.8053
Epoch 2/10
86/86 [==============================] - 765s 9s/step - loss: 0.2965 - accuracy: 0.8756 - val_loss: 0.1009 - val_accuracy: 0.9638
Epoch 3/10
86/86 [==============================] - 756s 9s/step - loss: 0.0819 - accuracy: 0.9753 - val_loss: 0.0683 - val_accuracy: 0.9785
Epoch 4/10
86/86 [==============================] - 754s 9s/step - loss: 0.0574 - accuracy: 0.9834 - val_loss: 0.0799 - val_accuracy: 0.9777
Epoch 5/10
86/86 [==============================] - 761s 9s/step - loss: 0.0588 - accuracy: 0.9828 - val_loss: 0.0548 - val_accuracy: 0.9832
Epoch 6/10
86/86 [==============================] - 767s 9s/step - loss: 0.0468 - accuracy: 0.9860 - val_loss: 0.0512 - val_accuracy: 0.9879
Epoch 7/10
86/86 [==============================] - 776s 9s/step - loss: 0.0430 - accuracy: 0.9868 - val_loss: 0.0512 - val_accuracy: 0.9879
Epoch 8/10
86

In [ ]:
# predict for test set
pred=model.predict(testX)  
# biểu thị xác suất để nó thuộc lớp dương 1

86/86 [==============================] - 47s 540ms/step


In [ ]:
# Threshold values predicted

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [ ]:
# Count True predicted and wrong predicted

true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1
        
print("correct predicted :: ", true)
print("false prediction :: ", false)

correct predicted ::  2703
false prediction ::  35


In [ ]:
# Number of attack and benign data in test set

attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)

Attack data in test set ::  1478
 Benign data in test set ::  1260


**Confusion Matrix**

In [ ]:
# Takes True positive, true negative, false positive and false negative as input
# Returns Accuracy

def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [ ]:
# Takes True Positive and false positive
# Returns Precision

def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [ ]:
#Takes True Positive and false negative
# Returns Recall

def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [ ]:
#Takes true values and predicted
# Returns Accuracy, precision and recall


def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [ ]:
accuracy,precision,recall=confusion_matrix(testY,pred)

In [ ]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.9959404600811907 
 Precision : 1.0 
 Recall : 0.9959404600811907


In [ ]:
from sklearn.metrics import classification_report
print("another: ")
print(classification_report(testY, pred))

another: 
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1260
           1       0.98      1.00      0.99      1478

    accuracy                           0.99      2738
   macro avg       0.99      0.99      0.99      2738
weighted avg       0.99      0.99      0.99      2738

